In [45]:
import numpy as np
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [2]:
def encode_symbols(symbols):
    # Make unique characters
    symbols = set(symbols)
    # Sort for logical ordering scheme
    symbols = sorted(symbols)
    sym_to_code = {sym:code for code,sym in enumerate(symbols)}
    code_to_sym = {code:sym for code,sym in enumerate(symbols)}
    return sym_to_code, code_to_sym

In [121]:
data = open('pokemon_names.txt', 'r').read()
names = [ name + '\0' for name in data.split('\n')]
char_to_code, code_to_char = encode_symbols(data + '\0')
# Constants
vocab_size = len(char_to_code)
n_hidden = 256
n_epochs = 1
seq_length = 1

In [140]:
dataY[0]

[8, 21, 19, 7, 25, 20, 21, 29, 0]

In [127]:
dataX = []
dataY = []

for i in range(len(names)):
    seq_in = names[i][0:-1]
    seq_out = names[i][1:]
    dataX.append([char_to_code[ch] for ch in seq_in])
    dataY.append([char_to_code[ch] for ch in seq_out])
    
n_train_seq = len(dataX)

In [139]:
# Reshape into [samples, train_seq, features]
#x = np.reshape(dataX, (n_train_seq, seq_length, 1))
x = dataX
# Normalize
#x = [x_i / float(vocab_size) for x_i in x]
# y into one hot 
y = [np_utils.to_categorical(y_i) for y_i in dataY]

ValueError: zero-size array to reduction operation maximum which has no identity

In [105]:
# LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(x.shape[1], x.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
opt = optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt)

In [106]:
# Define the checkpoints
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [107]:
# fit the model
model.fit(x, y, epochs=10, batch_size=64, callbacks=callbacks_list, shuffle=True)

Epoch 1/10
6047/6047 [==============================] - 8s 1ms/step - loss: 3.0516
Epoch 2/10
6047/6047 [==============================] - 1s 198us/step - loss: 2.9422
Epoch 3/10
6047/6047 [==============================] - 1s 191us/step - loss: 2.9071
Epoch 4/10
6047/6047 [==============================] - 1s 225us/step - loss: 2.8937
Epoch 5/10
6047/6047 [==============================] - 1s 187us/step - loss: 2.8831
Epoch 6/10
6047/6047 [==============================] - 1s 203us/step - loss: 2.8715
Epoch 7/10
6047/6047 [==============================] - 1s 195us/step - loss: 2.8651
Epoch 8/10
6047/6047 [==============================] - 1s 198us/step - loss: 2.8566
Epoch 9/10
6047/6047 [==============================] - 1s 184us/step - loss: 2.8556
Epoch 10/10
6047/6047 [==============================] - 1s 189us/step - loss: 2.8464


In [9]:
# Load Trained Weights
model.load_weights('filename')

OSError: Unable to open file (unable to open file: name = 'filename', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [114]:
def sample_text(sample_length):
    # Random Seed
    pattern = dataX[np.random.randint(0, len(dataX) - 1)]
    #print('Seed: ')
    #print(''.join([code_to_char[ch] for ch in pattern]))
    #print('---------------------------------')
    for i in range(sample_length):
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(vocab_size)
        pred = model.predict(x, verbose=0)
        sample_char = np.random.choice(range(vocab_size), p=pred.ravel())
        sample_char = np.argmax(pred)
        if sample_char == 0:
            print()
            break
        result = code_to_char[sample_char]
        sys.stdout.write(result)
        pattern.append(sample_char)
        pattern = pattern[1:len(pattern)]

In [119]:
for _ in range(10):
    sample_text(10)





sen



o



In [85]:
x.shape[2]

1